In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model

In [3]:
# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/My Drive/DL/Hackathon_FakeVsReal/train'
validation_data_dir = '/content/drive/My Drive/DL/Hackathon_FakeVsReal/test'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

In [10]:
datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.25) 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        subset="training",
        seed=42,
        shuffle=True,
        class_mode='binary')
 
validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        subset="validation",
        seed=42,
        shuffle=True,
        class_mode='binary')

Found 1282 images belonging to 2 classes.
Found 427 images belonging to 2 classes.


In [11]:
validation_data_dir = '/content/drive/My Drive/DL/Hackathon_FakeVsReal/test/'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
        )

Found 332 images belonging to 2 classes.


In [9]:
mnet = MobileNetV2(include_top = False, weights = "imagenet" ,input_shape=(96,96,3))

9420800/9406464 [==============================] - 0s 0us/step


In [14]:
import tensorflow as tf
tf.keras.backend.clear_session()

model_mnet = Sequential([mnet,
                    GlobalAveragePooling2D(),
                    Dense(512, activation = "relu"),
                    BatchNormalization(),
                    Dropout(0.3),
                    Dense(128, activation = "relu"),
                    Dropout(0.1),
                    # Dense(32, activation = "relu"),
                    # Dropout(0.3),
                    Dense(2, activation = "softmax")])

model_mnet.layers[0].trainable = False

model_mnet.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

model_mnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [15]:
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001 
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [16]:
hist = model_mnet.fit_generator(train_generator,
                    epochs=20,
                    callbacks=[lr_callbacks],
                    validation_data=validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
41/41 [==============================] - 440s 11s/step - loss: 0.8629 - accuracy: 0.5741 - val_loss: 0.7095 - val_accuracy: 0.5504 - lr: 0.0010
Epoch 2/20
41/41 [==============================] - 50s 1s/step - loss: 0.7355 - accuracy: 0.6373 - val_loss: 0.7883 - val_accuracy: 0.5597 - lr: 0.0010
Epoch 3/20
41/41 [==============================] - 50s 1s/step - loss: 0.6601 - accuracy: 0.6529 - val_loss: 0.8069 - val_accuracy: 0.5808 - lr: 0.0010
Epoch 4/20
41/41 [==============================] - 50s 1s/step - loss: 0.6232 - accuracy: 0.6958 - val_loss: 0.7079 - val_accuracy: 0.6183 - lr: 1.0000e-04
Epoch 5/20
41/41 [==============================] - 50s 1s/step - loss: 0.5827 - accuracy: 0.6958 - val_loss: 0.7542 - val_accuracy: 0.5738 - lr: 1.0000e-04
Epoch 6/20
41/41 [==============================] - 50s 1s/step - loss: 0.5565 - accuracy: 0.7246 - val_loss: 0.7204 - val_accuracy: 0.5948 - lr: 1.0000e-04
Epoch 7/20
41/41 [==============================] - 50s 1s/step - lo

In [17]:
test_predict1 = model_mnet.predict(test_generator)

In [18]:
test_predict1

array([[0.5156464 , 0.48435363],
       [0.27035445, 0.72964555],
       [0.17382741, 0.8261726 ],
       [0.7826337 , 0.21736626],
       [0.35038874, 0.64961123],
       [0.5449114 , 0.45508865],
       [0.09427951, 0.9057205 ],
       [0.66886294, 0.33113706],
       [0.80154824, 0.19845176],
       [0.38452443, 0.61547554],
       [0.19764733, 0.8023527 ],
       [0.82556075, 0.17443922],
       [0.9016651 , 0.09833486],
       [0.3027129 , 0.6972871 ],
       [0.30390045, 0.6960996 ],
       [0.7684464 , 0.23155358],
       [0.8137579 , 0.18624209],
       [0.4721168 , 0.52788323],
       [0.7705013 , 0.22949874],
       [0.45023683, 0.5497632 ],
       [0.91220856, 0.08779137],
       [0.12290932, 0.87709063],
       [0.80275905, 0.197241  ],
       [0.6841434 , 0.31585655],
       [0.04052729, 0.95947266],
       [0.220408  , 0.779592  ],
       [0.67338526, 0.3266147 ],
       [0.66140085, 0.33859915],
       [0.83896625, 0.16103378],
       [0.46797752, 0.5320225 ],
       [0.

In [19]:
labels1 = []
for i in range(len(test_predict1)):
  if test_predict1[i][0] <= 0.5 :
    labels1.append(0)
  else:
    labels1.append(1)

In [20]:
labels1

[1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,


In [21]:
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))

58900480/58889256 [==============================] - 0s 0us/step


In [22]:
vgg16_model.output[-1]


<KerasTensor: shape=(7, 7, 512) dtype=float32 (created by layer 'tf.__operators__.getitem')>

In [24]:
# model = Sequential()
# for layer in vgg16_model.layers[:-1]:
#     model.add(layer)

# for layer in model.layers:
#     layer.trainable = False
    
# model.add(Dense(2, activation='softmax'))


model_vgg = Sequential([vgg16_model,
                    Flatten(),
#                     GlobalAveragePooling2D(),
#                     Dense(512, activation = "relu"),
#                     BatchNormalization(),
#                     Dropout(0.3),
#                     Dense(128, activation = "relu"),
#                     Dropout(0.1),
#                     # Dense(32, activation = "relu"),
#                     # Dropout(0.3),
                    Dense(2, activation = "softmax")])

model_vgg.layers[0].trainable = False

model_vgg.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

model_vgg.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
hist =  model_vgg.fit_generator(train_generator,
                    epochs=20,
                    callbacks=[lr_callbacks],
                    validation_data=validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
41/41 [==============================] - 605s 15s/step - loss: 1.1723 - accuracy: 0.5468 - val_loss: 0.7848 - val_accuracy: 0.5972 - lr: 0.0010
Epoch 2/20
41/41 [==============================] - 656s 16s/step - loss: 0.7526 - accuracy: 0.5928 - val_loss: 0.8882 - val_accuracy: 0.5808 - lr: 0.0010
Epoch 3/20
41/41 [==============================] - 657s 16s/step - loss: 0.8079 - accuracy: 0.6030 - val_loss: 0.8070 - val_accuracy: 0.5785 - lr: 0.0010
Epoch 4/20
41/41 [==============================] - 606s 15s/step - loss: 0.6446 - accuracy: 0.6755 - val_loss: 0.7416 - val_accuracy: 0.6112 - lr: 1.0000e-04
Epoch 5/20
41/41 [==============================] - 657s 16s/step - loss: 0.6068 - accuracy: 0.7051 - val_loss: 0.7794 - val_accuracy: 0.5925 - lr: 1.0000e-04
Epoch 6/20
41/41 [==============================] - 657s 16s/step - loss: 0.5709 - accuracy: 0.7114 - val_loss: 0.7439 - val_accuracy: 0.5972 - lr: 1.0000e-04
Epoch 7/20
41/41 [==============================] - 656s 1